In [1]:
# Jupyter notebook for testing purposes. Will create formal julia analysis script later. 

include("utilities.jl");
default(show=false)
default(fontfamily="Computer Modern")

In [2]:
# HYPERPARAMS
n_runs = 10
patterns = 0:1
duration = 750
labels = ["theta" , "ftheta", "alpha", "beta", "gamma"]
freqlabels = [L"\theta" L"\theta_{fast}" L"\alpha" L"\beta" L"\gamma"]

fig_ext = ".png"

# CREATE NECESSARY DIRECTORIES 
create_directories_if_not_exist()

# IDENTIFY NEURON POPULATION RANGES
populations = Dict(
    "DG" => [0,1]
)

Dict{String, Vector{Int64}} with 1 entry:
  "DG" => [0, 1]

In [3]:
function load_spike_files(
    patterns::Union{UnitRange{Int64}, Vector{Int64}}, 
    model_label::String,
    neuron_ids::Dict;
    neurons_per_pattern::Int64=1, #
    data_dir::String="data/gcpp/")

    df = DataFrame()
    for p ∈ patterns
        spike_fname = data_dir*"DGsp-"*string(p)*"-"*string(neurons_per_pattern)*"-"*model_label*".txt"
        stims_fname = data_dir*"StimIn-"*string(p)*"-"*string(neurons_per_pattern)*"-"*model_label*".txt"
        spikes = CSV.read(spike_fname, delim="\t", header=0, DataFrame)
        stimin = CSV.read(stims_fname, delim="\t", header=0, DataFrame)

        if size(spikes, 1) > 0
            rename!(spikes, ["Time", "Neuron"])
            spikes[:,"Population"] .= "" 
            spikes[:,"Pattern"] .= p
            spikes[:,"NeuronsPerPattern"] .= neurons_per_pattern 
            spikes[:,"Model"] .= model_label
            for k ∈ keys(neuron_ids) 
                lb, ub = neuron_ids[k]
                spikes[lb .<= spikes[:, "Neuron"] .< ub, "Population"] .= k
            end

            df = [df; spikes]
        end 

        if size(stimin, 1) > 0
            rename!(stimin, ["Time", "Neuron"])
            stimin[:,"Population"] .= "PP"
            stimin[:,"Pattern"] .= p
            stimin[:,"NeuronsPerPattern"] .= neurons_per_pattern 
            stimin[:,"Model"] .= model_label
            df = [df; stimin]
        end
    end

    return df
end

load_spike_files (generic function with 1 method)

In [18]:
# ISI figure

default(fontfamily="Computer Modern")
n_runs = 10
save_cov = OrderedDict("theta"=>[], "ftheta"=>[], "alpha"=>[], "beta"=>[], "gamma"=>[])

for run_ ∈ 1:n_runs
    for l ∈ 1:length(labels)
        spikes = load_spike_files(patterns, labels[l]*"-$run_", populations)
        gc = spikes[(spikes.Population .== "DG") .& (spikes.Pattern .== 1), :]
        spiketimes = gc[!,"Time"]
        ISIs = []

        for i ∈ 1:(length(spiketimes)-1)
            time1 = spiketimes[i]
            time2 = spiketimes[i+1] 
            ISI = time2 - time1 
            append!(ISIs, ISI)
        end

        cov = mean(ISIs)
        append!(save_cov[labels[l]], cov)

    end
end

meanisi = OrderedDict("theta"=>[], "ftheta"=>[], "alpha"=>[], "beta"=>[], "gamma"=>[])
stdisi = OrderedDict("theta"=>[], "ftheta"=>[], "alpha"=>[], "beta"=>[], "gamma"=>[])

for l ∈ 1:length(labels)
    meanv = []
    stdv = []
    meanv = mean(save_cov[labels[l]])
    append!(meanisi[labels[l]], meanv)
    stdv =  std(save_cov[labels[l]])
    append!(stdisi[labels[l]], stdv)
end


unpack(a) = eltype(a[1])[el[1] for el in a]

avgisi = unpack(collect(values(meanisi)))
varisi = unpack(collect(values(stdisi)))
freqlabels = [L"\theta", L"\theta_{fast}", L"\alpha", L"\beta", L"\gamma"]

isi_fig = plot(freqlabels, avgisi, yerror = varisi, xlabel = "Input Frequency Band", 
            xtickfont=font(18), xguidefontsize=16, ytickfontsize = 16, yguidefontsize = 16, 
            markerstrokewidth =2, grid=:none, ylabel = "Mean ISI (ms)", 
    c = :black, linewidth = 4, dpi=300, size=(350,350), label=nothing)

savefig(isi_fig, "figures/pattern-separation/ISI_10runs-CCN"*fig_ext)


In [222]:
# VOLTAGE TRACE + INPUT SPIKE TRAIN FIGURE
volt_fname = "data/gcpp/DGVt-1-1-gamma-1.txt"
volttrace = CSV.read(volt_fname, delim="\t", header=1, DataFrame)
volttrace = volttrace[:, 1:2]

spikes = load_spike_files(patterns, labels[5]*"-1", populations)
stimin = spikes[(spikes.Population .== "PP") .& (spikes.Pattern .== 1), :]
plots = []

rename!(volttrace, ["Time", "Voltage"])

p = scatter(stimin[:,1], stimin[:,2], 
            marker = (:vline, 30, 0.8, :red, stroke(4, 0.8, :black, :vline)),
                label=nothing, c=:red, msc=:black, #msa=1, ma=1, 
                xlims = [0,285], ylims = [0,5], axis=([], false), grid = false )

vtrace = plot(volttrace[1:2850, "Time"], volttrace[1:2850, "Voltage"],
        color = :black, linewidth = 2, legend = :none, axis=([], false), grid = false, 
        size = (600,300))

append!(plots, [vtrace])
append!(plots, [p])

fig = plot(plots..., layout=grid(2, 1, heights=[0.95, 0.05]), size=(500, 400), dpi = 300)

savefig(fig, "figures/voltage-tracings/voltage-tracings-gamma.png")



┌ Warning: thread = 1 warning: parsed expected 2 columns, but didn't reach end of line around data row: 1. Parsing extra columns and widening final columnset
└ @ CSV /Users/selenasingh/.julia/packages/CSV/jFiCn/src/file.jl:579
